In [42]:
from openai import OpenAI
import json

client = OpenAI()


tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the weather for a specific location.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and country to look up, e.g., 'Berlin, Germany'"
                    }
                },
                "required": ["location"]
            }
        }
    }
]

# --- Define your actual function ---
def get_weather(location):
    return f"The weather in {location} is sunny and 25°C."  # Mock implementation

# --- ReAct Agent Loop using tool calling ---

user_question = "Test to use the tool"
messages = [
    {"role": "system", "content": (
        "You are a helpful agent that reasons step-by-step to solve problems. "
        "Use the format:\n"
        "Thought: your reasoning\n"
        "Action: function_name(args)\n"
        "Observation: result from tool\n"
        "Repeat until final answer.\n"
        "End with:\nAnswer: your final answer."
    )},
    {"role": "user", "content": user_question}
]

trace = []

while True:
    # Call OpenAI with tool calling
    response =  client.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )

    msg = response.choices[0].message
    messages.append(msg)

    # Add to trace
    if msg.content:
        print(msg["content"])
        trace.append(msg["content"])

    # If a tool call is suggested
    if "tool_calls" in msg:
        for tool_call in msg["tool_calls"]:
            name = tool_call["function"]["name"]
            args = json.loads(tool_call["function"]["arguments"])
            tool_id = tool_call["id"]

            if name == "get_weather":
                result = get_weather(**args)

            # Append tool result as observation
            tool_response = {
                "role": "tool",
                "tool_call_id": tool_id,
                "content": result
            }
            messages.append(tool_response)

            # Show trace
            observation = f"Observation: {result}"
            print(observation)
            trace.append(observation)

    # If final answer given (no tool calls)
    elif msg.content and not msg.tool_calls:
        break

trace


BadRequestError: Error code: 400 - {'error': {'message': "An assistant message with 'tool_calls' must be followed by tool messages responding to each 'tool_call_id'. The following tool_call_ids did not have response messages: call_y0sydr4yVcraHV6MGVNWeric, call_NH6cjNmHjCLQvho7FYOKF0UN", 'type': 'invalid_request_error', 'param': 'messages', 'code': None}}